In [10]:
import io
import requests
from pytesseract import pytesseract
from PIL import Image
from urllib.request import urlopen
from urllib.error import HTTPError
import pandas as pd

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = path_to_tesseract

In [11]:
posts = pd.read_csv("data/Applaudo Studios posts - Applaudo Studios posts.csv")
posts = posts.drop('Unnamed: 0', axis=1)
posts.head()

,post_url,author,author_url,author_type,description,media_type,media_url
0,https://www.linkedin.com/feed/update/urn:li:ac...,Universidad Francisco Gavidia,https://www.linkedin.com/company/ufgoficial/,Company,Check these open positions at Applaudo Studios...,Carousel,['https://media-exp3.licdn.com/dms/image/C4D1F...
1,https://www.linkedin.com/feed/update/urn:li:ac...,Jose Danilo Cortez Orellana,https://www.linkedin.com/in/jose-danilo-cortez...,User,Check this HR opportunity at Applaudo Studios .,Shared LinkedIn post,https://www.linkedin.com/feed/update/urn:li:ac...
2,https://www.linkedin.com/feed/update/urn:li:ac...,Darwin Romero,https://www.linkedin.com/in/darwin-romero-2a30...,User,We continue to accelerate our Development-As-A...,Single image,https://media-exp3.licdn.com/dms/image/C4E22AQ...
3,https://www.linkedin.com/feed/update/urn:li:ac...,Applaudo Studios,https://www.linkedin.com/company/applaudo-stud...,Company,Today we want you to meet another incredible t...,Video,https://dms.licdn.com/playlist/C4E05AQHJfjKSiW...
4,https://www.linkedin.com/feed/update/urn:li:ac...,César Retana,https://www.linkedin.com/in/cretana,User,Today I got my first Performance Review delive...,Other,NaN


In [12]:
def get_text(url):
    img = Image.open(urlopen(url))
    text = pytesseract.image_to_string(img)
    return text

In [17]:
text_list = []
for i,row in posts.iterrows():
    print(i)
    text = ''
    if row['media_type'] == 'Single image':
        text = get_text(row['media_url'])

    elif row['media_type'] in ['Multiple images','Carousel']:
        for img in row['media_url'].replace('[','').replace(']','').replace("'",'').split(','):
            if type(img) == 'None':
                continue
            try:
                text += '\n' + get_text(img)
            except HTTPError as err:
                print(err, img)
            except:
                continue

    text_list.append(text)

0
HTTP Error 403: Forbidden https://media-exp3.licdn.com/dms/image/C4D1FAQEfNOb1DM4-Gg/feedshare-document-images_480/1/1625675311597?e=1626217200&v=beta&t=JANdkEtnbaAkOK1jYXVAWyF23LP_kB7YZCnjpovXk9Q
HTTP Error 403: Forbidden  https://media-exp3.licdn.com/dms/image/C4D1FAQEfNOb1DM4-Gg/feedshare-document-images_160/2/1625675311541?e=1626217200&v=beta&t=UrgNdy-yMPYKdb0XFH27XL3hHgb3BfyQpKIosbUWjjw
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
HTTP Error 403: Forbidden https://media-exp3.licdn.com/dms/image/C4D1FAQE_iqlN3LQ7KQ/feedshare-document-images_480/1/1621873134370?e=1626224400&v=beta&t=IXazkLAO70sfUkvt5vL6WXV4EPNL3RlnH8-HYmOU2dw
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
HTTP Error 403: Forbidden https://media-exp1.licdn.com/dms/image/C4D1FAQEdaj8Uw0TLxQ/feeds

In [18]:
try:
    posts = posts.drop('media_text', axis=1)
except:
    print()

posts['media_text'] = text_list

In [19]:
posts.to_csv('Applaudo Studios posts - Applaudo Studios posts - Media Text.csv')